DPT
===

**Vision Transformers for Dense Prediction**

 * Paper: https://arxiv.org/abs/2103.13413

![DPT](../assets/dpt_overview.png)

```bash
pip install torch torchvision
pip install transformers
```

In [1]:
from PIL import Image
import numpy as np
import torch
from transformers import DPTImageProcessor
from transformers import DPTForDepthEstimation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_processor = DPTImageProcessor.from_pretrained(
    "Intel/dpt-hybrid-midas"
)
model = DPTForDepthEstimation.from_pretrained(
    "Intel/dpt-hybrid-midas", low_cpu_mem_usage=True
)

model.eval().to(device);